In [25]:
import tensorflow as tf
import numpy as np
import cv2

In [26]:
tf.reset_default_graph()

img_height=480
img_width=360
#Network Parameters
n_input=img_height*img_width*3
batch_size=1
display_step=1
dropout=0.8
epoch=2

#tf graph input
x=tf.placeholder(tf.float32,[None,img_height,img_width,3])

keep_prob=tf.placeholder(tf.float32)

In [27]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,output_shape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=output_shape,strides=[1,1,1,1],padding='SAME'),b))
def cv_bottleneck(img,w1,w2):
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d(img,w1,strides=[1,1,1,1],padding='SAME'))
    tmp_bn_2=tf.nn.conv2d(tmp_bn_1,w2,strides=[1,1,1,1],padding='SAME')
    return (img+tmp_bn_2)
def dc_bottleneck(img,w1,w2,output_shape):
    output_shape[3]=w1.get_shape().as_list()[2]
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d_transpose(img,w1,output_shape=output_shape,strides=[1,1,1,1],padding='SAME'))
    output_shape[3]=w2.get_shape().as_list()[2]
    tmp_bn_2=tf.nn.conv2d_transpose(tmp_bn_1,w2,output_shape=output_shape,strides=[1,1,1,1],padding='SAME')
    return (img+tmp_bn_2)
def hourglass(img,wc1_1,wc1_2,bc1,wc2_1,wc2_2,bc2,wc3_1,wc3_2,bc3,wc4_1,wc4_2,bc4,wc5_1,wc5_2,bc5):
    tmp_cv_1=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(img,wc1_1,wc1_2),bc1))
    tmp_cv_1_pooling=max_pool(tmp_cv_1,k=2)
    tmp_cv_1_pooling=tf.nn.dropout(tmp_cv_1_pooling,keep_prob)
    
    tmp_cv_2=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_cv_1_pooling,wc2_1,wc2_2),bc2))
    tmp_cv_2_pooling=max_pool(tmp_cv_2,k=2)
    tmp_cv_2_pooling=tf.nn.dropout(tmp_cv_2_pooling,keep_prob)
    
    tmp_cv_3=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_cv_2_pooling,wc3_1,wc3_2),bc3))
    tmp_cv_3=tf.nn.dropout(tmp_cv_3,keep_prob)
    
    tmp_rs_1=tf.image.resize_bilinear(tmp_cv_3,tmp_cv_2.get_shape().as_list()[1:3])
    tmpShape=tmp_cv_1_pooling.get_shape().as_list()
    tmpShape[0]=batch_size
    tmp_dc_1=tf.nn.relu(tf.nn.bias_add(dc_bottleneck(tmp_rs_1,wc4_1,wc4_2,output_shape=tmpShape),bc4))
    
    tmp_add_1=tf.add(tmp_dc_1,tmp_cv_1_pooling)
    tmp_add_1=tf.nn.dropout(tmp_add_1,keep_prob)
    
    tmp_rs_2=tf.image.resize_bilinear(tmp_add_1,tmp_cv_1.get_shape().as_list()[1:3])
    tmpShape=img.get_shape().as_list()
    tmpShape[0]=batch_size
    tmp_dc_2=tf.nn.relu(tf.nn.bias_add(dc_bottleneck(tmp_rs_2,wc5_1,wc5_2,output_shape=tmpShape),bc5))
    
    tmp_add_2=tf.add(tmp_dc_2,img)
    tmp_add_21=tf.nn.dropout(tmp_add_2,keep_prob)
    
    return (tmp_add_2)

In [28]:
wc1=tf.Variable(tf.random_normal([7,7,3,16]),name="wc1")
bc1=tf.Variable(tf.random_normal([16]),name="bc1")

wh1_1_1=tf.Variable(tf.random_normal([5,5,16,64]),name="wh1_1_1")
wh1_1_2=tf.Variable(tf.random_normal([5,5,64,16]),name="wh1_1_2")

bh1_1=tf.Variable(tf.random_normal([16]),name="bh1_1")

wh1_2_1=tf.Variable(tf.random_normal([3,3,16,64]),name="wh1_2_1")
wh1_2_2=tf.Variable(tf.random_normal([3,3,64,16]),name="wh1_2_2")

bh1_2=tf.Variable(tf.random_normal([16]),name="bh1_2")

wh1_3_1=tf.Variable(tf.random_normal([3,3,16,16]),name="wh1_3_1")
wh1_3_2=tf.Variable(tf.random_normal([3,3,16,16]),name="wh1_3_2")

bh1_3=tf.Variable(tf.random_normal([16]),name="bh1_3")

wh1_4_1=tf.Variable(tf.random_normal([3,3,64,16]),name="wh1_4_1")
wh1_4_2=tf.Variable(tf.random_normal([3,3,16,64]),name="wh1_4_2")

bh1_4=tf.Variable(tf.random_normal([16]),name="bh1_4")

wh1_5_1=tf.Variable(tf.random_normal([3,3,64,16]),name="wh1_5_1")
wh1_5_2=tf.Variable(tf.random_normal([3,3,16,64]),name="wh1_5_2")

bh1_5=tf.Variable(tf.random_normal([16]),name="bh1_5")

wd1=tf.Variable(tf.random_normal([7,7,16,16]),name="wd1")
bd1=tf.Variable(tf.random_normal([16]),name="bd1")

wc2=tf.Variable(tf.random_normal([1,1,16,16]),name="wc2")
bc2=tf.Variable(tf.random_normal([16]),name="bc2")

In [29]:
#Construct model
conv1=conv2d(x,wc1,bc1)
conv1_pooling=max_pool(conv1,k=2)
conv1_pooling=tf.nn.dropout(conv1_pooling,keep_prob)

hg_1=hourglass(conv1_pooling,wh1_1_1,wh1_1_2,bh1_1,wh1_2_1,wh1_2_2,bh1_2,wh1_3_1,wh1_3_2,bh1_3,wh1_4_1,wh1_4_2,bh1_4,wh1_5_1,wh1_5_2,bh1_5)

resize1=tf.image.resize_bilinear(hg_1,conv1.get_shape().as_list()[1:3])
tmpShape=x.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=wd1.get_shape().as_list()[2]
dconv1=conv2d_transpose(resize1,wd1,bd1,output_shape=tmpShape)
dconv1=tf.nn.dropout(dconv1,keep_prob)

conv2=conv2d(dconv1,wc2,bc2)
conv2=tf.nn.dropout(conv2,keep_prob)


In [30]:
saver = tf.train.Saver()
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [31]:
pairs=[[7,8],[8,9],[6,5],[5,4],[6,0],[7,0],[0,1],[1,2],[2,3],[10,11],[11,12],[15,14],[14,13],[12,1],[13,1]]
def draw_bones(img,joints):
    for line in pairs:
        a=(int(joints[line[0]][0]),int(joints_pos[line[0]][1]))
        b=(int(joints[line[1]][0]),int(joints_pos[line[1]][1]))
        cv2.line(img,a,b,(255,0,0),5)
    return img

In [32]:
with tf.Session(config=config) as sess:
    sess.run(init)
    saver.restore(sess,"model/model_hg.ckpt")
    cap = cv2.VideoCapture(0)
    
    np_img=np.zeros([1,img_height,img_width,3])
    joints_pos=np.zeros([16,2])
    #tmpCount=0
    #while tmpCount<1:
    while True:
        ret, frame = cap.read()
        #print(type(frame[0:479,140:499]))
        np_img[0]=np.array(frame[0:480,140:500])
        output=sess.run(conv2,feed_dict = {x:np_img,keep_prob:1.})
        for i in range(0,16):
            tmp_heatmap=output[0,:,:,i]
            single_joint_location = np.unravel_index(tmp_heatmap.argmax(), tmp_heatmap.shape)
            joints_pos[i]=single_joint_location
        frame=draw_bones(frame,joints_pos)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from model/model_hg.ckpt
